In [27]:
from glob import glob
from os.path import join, split, splitext, dirname
import pandas as pd
from scipy.stats import norm
import numpy as np
from functools import partial
import matplotlib.pyplot as plt
from scipy import stats

In [17]:
base_path = '/home/rgiordan/Documents/git_repos/DADVI/dadvi-experiments/'
input_path = join(base_path, 'comparison/analysis/coverage_warm_starts_rerun')
output_path = join(base_path, 'comparison/analysis/coverage_warm_starts_rerun')

M_vals = [ 8, 16, 32, 64 ]

In [96]:
M = 8
coverage_filenames = glob(join(input_path, f'M_{M}', '*', '*.pkl'))
raw_model_df = pd.read_pickle(filename)
print(raw_model_df.keys())
#raw_model_df['means'] - raw_model_df['reference_means'] # They are different
reference_means = raw_model_df['reference_means'].iloc[0]
reference_freq_sds = raw_model_df['reference_freq_sds'].iloc[0]
assert(len(reference_means) == param_dim)
reference_freq_sds

Index(['means', 'seed', 'freq_sds', 'newton_step_norm', 'scipy_opt_result',
       'reference_means', 'reference_freq_sds', 'M',
       'reference_newton_step_norm', 'reference_scipy_opt_result'],
      dtype='object')


array([0.27002875, 0.10808994, 0.3300278 , 0.24688348, 0.29446138,
       0.34176003, 0.18110268, 0.2904307 , 0.23119037, 0.26523586,
       0.27002209, 0.33879955, 0.26460352, 0.20790123, 0.30023867,
       0.39132133, 0.27248412, 0.19481111, 0.09495773, 0.26530274,
       0.29544409, 0.36691676, 0.3257251 , 0.23717544, 0.21085609,
       0.08502773, 0.26981055, 0.28387719, 0.34385135, 0.21893104,
       0.28130681, 0.28169805, 0.30902894, 0.30092538, 0.23783169,
       0.32397374, 0.23184034, 0.27316882, 0.27181886, 0.28467147,
       0.24033208, 0.34773134, 0.2415593 , 0.24559177, 0.21755859,
       0.29930083, 0.37634131, 0.3102693 , 0.25291185, 0.33942954,
       0.27611983, 0.31690842, 0.32969616, 0.15585287, 0.24495493,
       0.21267922, 0.2701069 , 0.31545576, 0.26422724, 0.32331489,
       0.15756097, 0.36932149, 0.30279407, 0.21078926, 0.46802372,
       0.18649955, 0.21614616, 0.26115626, 0.29029963, 0.07426278,
       0.15475933, 0.20063362, 0.3632832 , 0.29093577, 0.29958

In [97]:
def RepList(x, n):
    return [x for _ in range(n)]


def GetCoverageDataframe(filename, M):
    raw_model_df = pd.read_pickle(filename)

    # The final directory is the model name
    model_name = split(os.path.dirname(filename))[-1]

    num_runs = raw_model_df.shape[0]

    means = raw_model_df['means'].to_numpy()
    seeds = raw_model_df['seed'].to_numpy()
    freq_sds = raw_model_df['freq_sds'].to_numpy()
    assert(len(means) == len(freq_sds))
    num_runs = len(means) 
    param_dim = len(means[0]) 

    # A for loop is not the most efficient or elegant but it will let me make sure
    # everything lines up correctly
    model_dict = {
        'seed': [],
        'param': [],
        'mean': [],
        'freq_sd': []
    }
    param_dims = np.arange(param_dim)
    for i in range(num_runs):
        assert(len(means[i]) == len(freq_sds[i]))
        model_dict['seed'].append(RepList(seeds[i], param_dim))
        model_dict['param'].append(param_dims)
        model_dict['mean'].append(means[i])
        model_dict['freq_sd'].append(freq_sds[i])

    # Save the ''reference'' as well
    reference_means = raw_model_df['reference_means'].iloc[0]
    reference_freq_sds = raw_model_df['reference_freq_sds'].iloc[0]
    assert(len(reference_means) == param_dim)
    assert(len(reference_means) == len(reference_freq_sds))
    model_dict['seed'].append(RepList('reference', param_dim))
    model_dict['param'].append(param_dims)
    model_dict['mean'].append(reference_means)
    model_dict['freq_sd'].append(reference_freq_sds)
        
    model_df = pd.DataFrame()
    for k,v in model_dict.items():
        model_df[k] = np.hstack(v)
    model_df['model'] = model_name
    model_df['num_draws'] = M
    
    return model_df

In [112]:
model_dfs = []
for M in M_vals:
    print(f'Loading for {M} draws')
    coverage_filenames = glob(join(input_path, f'M_{M}', '*', '*.pkl'))
    assert(len(coverage_filenames) > 0)
    model_dfs.append(
        pd.concat([GetCoverageDataframe(filename, M) for filename in coverage_filenames]))

model_df = pd.concat(model_dfs)

Loading for 8 draws
Loading for 16 draws
Loading for 32 draws
Loading for 64 draws


In [113]:
np.unique(model_df['num_draws'].to_numpy())

array([ 8, 16, 32, 64])

In [114]:
print(output_path)
model_df.to_csv(join(output_path, 'coverage_tidy.csv'), index=False)

/home/rgiordan/Documents/git_repos/DADVI/dadvi-experiments/comparison/analysis/coverage_warm_starts_rerun
